In [13]:
import pandas as pd

In [14]:
# sql calls sa production
preference_df = pd.read_csv("user_preference.csv")
anime_df = pd.read_csv("anime.csv")
top5_peruser_df = pd.read_csv("top5_amime_per_user.csv") # per crop order
ratings_df = pd.read_csv("rating.csv")

In [9]:
def get_similar_users(preference_df, user_id):
    """
    Gets 10 users with the most similar preference from the given user. Uses hamming distance for this, where
    the lower the hamming score, the more similar they are.
    
    Input: preference_df - dataframe
    Output: dataframe with the top 5 most similar users
    """
    
    user_pref = preference_df.loc[preference_df.user_id == user_id].preference.values[0]
    users_sample = preference_df.sample(n=1000, random_state=420)
    users_sample['hamming_score'] = preference_df.preference.apply(
                                    lambda val: sum(c1 != c2 for c1, c2 in zip(val, user_pref)))
    return users_sample.sort_values("hamming_score")[:10]

In [10]:
def get_recommended_anime(user_id, preference_df, top5_peruser_df, anime_df, extensive=False):
    """
    Gets 20 recommeded anime for a certain user.
    
    Input: user_id (int)
           preference_df (dataframe) - preference dataframe mined in previous notebook
           top5_peruser_df (dataframe) - top 5 animes per user mined in previous notebook
           anime_df - dataframe - anime_df in the original dataset
           
    Output: dataframe with 20 recommended animes
    """
    
    # getting top 5 users with similar prefrences
    similar_users = get_similar_users(preference_df, user_id)
    
    # getting watched anime of similar users
    if extensive is True:
        # top 5 animes (max) per user
        # optional: to add randomizing factor
        anime_ids = similar_users.merge(top5_peruser_df, how="left")
    else:
        # gets all of the animes watched by similar users with a rating >= 7
        anime_ids = similar_users.merge(ratings_df, how="left")
        anime_ids = anime_ids[anime_ids.rating >= 7][["anime_id"]]
    
    watched_by_similar = anime_ids[["anime_id"]].merge(anime_df, 
                                      how="left").drop_duplicates().sort_values(by="rating", ascending=False)
    
    # getting user's watched anime, used to remove possibility of recommending anime that user has watched
    watched_anime = ratings_df[ratings_df.user_id == user_id][["anime_id"]]
    anime_recomms = watched_anime.merge(watched_by_similar, how="outer", on="anime_id", indicator=True)
    
    # get top 20 recommended anime
    return anime_recomms[anime_recomms["_merge"] == "right_only"][["name", "genre", 
                                                               "type", "episodes"]].reset_index()[:20]

In [3]:
from time import perf_counter as pf

In [4]:
Preferences:
- Take into consideration current data

1. find 10 users with similar profile as current user within a sample
(select * from table randomizer limit 500;)

2. find 5 top rated animes per user and add to list

3. remove animes that have been watched by current user from the list of all top rated animes

4. get the top 10 highest rated anime as a recommendation to the user

SyntaxError: invalid syntax (452158039.py, line 1)

In [5]:
ERD
- not subject to change
- static once naa sa db
- constant unless if naay mistake; open to CRUD

- rainfall
- soil type*
- soil texture*
- soil structure*
- distance per plant*
- average yield

SyntaxError: invalid syntax (2040254911.py, line 2)

In [68]:
farmer - location -> soil type == profile ni farmer

1.1. given sa soil profile ni farmer, unsay crop nga angay itanom sa soil profile

2.1. find other farmers with similar profile as farmer (tanan nga farmers (100), sample sa farmers if dako na kaayo)
2.2. find crops nga gitanom sa similar farmers or duol2 sa soil profile ni farmer

3 [join ang recomms 1.1 ug 1.2]

4. find crops nga preordered crops nga naa sa recommendation
* given sa yield (kg), find land area nga recommended/standard
* distance between plants/density
    - dense -> high competition for nutrients -> increase care (water, fertilizer)
    - (distance_per_plant (crop) * (expected_yield(kg sa order) / 
                                    average_yield_per_plant(db) )) = recommended_area (crop)
    * calculate beforehand
        - sort by ascending recommended_area (masigo, gamay UI label (best fit for you), 
                                              then katong uban recomm pero wala katong label) 
    - label (*most suitable == land area sobra or equal to the recommended), recommend (more dense)
    
- land ni farmer masigo sa katong preordered crop

- TBD: muni, province, region sa FE

5. give to farmer

12294

In [ ]:
more data == better results
70 > 2
100  100

equation of a line

In [6]:
get_recommended_anime(2666, preference_df, top5_peruser_df, anime_df)

NameError: name 'preference_df' is not defined

In [58]:
get_recommended_anime(2666, preference_df, top5_peruser_df, anime_df)

,index,name,genre,type,episodes
0,110,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26
1,111,Suzumiya Haruhi no Shoushitsu,"Comedy, Mystery, Romance, School, Sci-Fi, Supe...",Movie,1
2,112,Mononoke Hime,"Action, Adventure, Fantasy",Movie,1
3,113,Monogatari Series: Second Season,"Comedy, Mystery, Romance, Supernatural, Vampire",TV,26
4,114,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",TV,27
5,115,Natsume Yuujinchou Shi,"Drama, Fantasy, Shoujo, Slice of Life, Superna...",TV,13
6,116,Howl no Ugoku Shiro,"Adventure, Drama, Fantasy, Romance",Movie,1
7,117,Kara no Kyoukai 5: Mujun Rasen,"Action, Drama, Mystery, Romance, Supernatural,...",Movie,1
8,118,Zoku Natsume Yuujinchou,"Drama, Fantasy, Shoujo, Slice of Life, Superna...",TV,13
9,119,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,"Drama, Slice of Life, Supernatural",TV,11


In [65]:
# ffa, fcfs
get_recommended_anime(2666, preference_df, top5_peruser_df, anime_df, extensive=True)

,index,name,genre,type,episodes
0,110,Mononoke Hime,"Action, Adventure, Fantasy",Movie,1
1,111,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,"Drama, Slice of Life, Supernatural",TV,11
2,112,Hotaru no Haka,"Drama, Historical",Movie,1
3,113,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26
4,114,Mahou Shoujo Madoka★Magica Movie 2: Eien no Mo...,"Drama, Magic, Psychological, Thriller",Movie,1
5,115,Toradora!,"Comedy, Romance, School, Slice of Life",TV,25
6,116,Danshi Koukousei no Nichijou,"Comedy, School, Shounen, Slice of Life",TV,12
7,117,Mahou Shoujo Madoka★Magica Movie 1: Hajimari n...,"Drama, Magic, Psychological, Thriller",Movie,1
8,118,Majo no Takkyuubin,"Adventure, Comedy, Drama, Fantasy, Magic, Romance",Movie,1
9,119,Kurenai no Buta,"Action, Adventure, Comedy, Drama, Historical, ...",Movie,1


In [59]:
# ffa, fcfs
get_recommended_anime(2666, preference_df, top5_peruser_df, anime_df, extensive=True)

,index,name,genre,type,episodes
0,110,Mononoke Hime,"Action, Adventure, Fantasy",Movie,1
1,111,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,"Drama, Slice of Life, Supernatural",TV,11
2,112,Hotaru no Haka,"Drama, Historical",Movie,1
3,113,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26
4,114,Mahou Shoujo Madoka★Magica Movie 2: Eien no Mo...,"Drama, Magic, Psychological, Thriller",Movie,1
5,115,Toradora!,"Comedy, Romance, School, Slice of Life",TV,25
6,116,Danshi Koukousei no Nichijou,"Comedy, School, Shounen, Slice of Life",TV,12
7,117,Mahou Shoujo Madoka★Magica Movie 1: Hajimari n...,"Drama, Magic, Psychological, Thriller",Movie,1
8,118,Majo no Takkyuubin,"Adventure, Comedy, Drama, Fantasy, Magic, Romance",Movie,1
9,119,Kurenai no Buta,"Action, Adventure, Comedy, Drama, Historical, ...",Movie,1


In [47]:
get_recommended_anime(10000, preference_df, top5_peruser_df, anime_df)

,index,name,genre,type,episodes
0,29,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64
1,30,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1
2,31,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,"Drama, Slice of Life, Supernatural",TV,11
3,32,Shokugeki no Souma,"Ecchi, School, Shounen",TV,24
4,33,Kuroko no Basket 2nd Season,"Comedy, School, Shounen, Sports",TV,25
5,34,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25
6,35,Toradora!,"Comedy, Romance, School, Slice of Life",TV,25
7,36,Rurouni Kenshin: Meiji Kenkaku Romantan,"Action, Adventure, Comedy, Historical, Romance...",TV,94
8,37,Fate/stay night: Unlimited Blade Works,"Action, Fantasy, Magic, Shounen, Supernatural",TV,12
9,38,Nanatsu no Taizai,"Action, Adventure, Ecchi, Fantasy, Shounen, Su...",TV,24


In [48]:
get_recommended_anime(10000, preference_df, top5_peruser_df, anime_df, extensive=True)

,index,name,genre,type,episodes
0,29,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64
1,30,Steins;Gate,"Sci-Fi, Thriller",TV,24
2,31,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24
3,32,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25
4,33,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1
5,34,Haikyuu!! Second Season,"Comedy, Drama, School, Shounen, Sports",TV,25
6,35,Shigatsu wa Kimi no Uso,"Drama, Music, Romance, School, Shounen",TV,22
7,36,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25
8,37,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,"Action, Drama, Historical, Martial Arts, Roman...",OVA,4
9,38,Hajime no Ippo,"Comedy, Drama, Shounen, Sports",TV,75


In [27]:
from random import choice

users = preference_df.user_id.tolist()
list_users = {}

while len(list_users) != 500:
    list_users[choice(users)] = 1

In [28]:
avg_time = 0

for item in list_users.keys():
    start = pf()
    get_recommended_anime(item, preference_df, top5_peruser_df, anime_df)
    avg_time += (pf() - start)
    
avg_time / 500

0.35814554676000443

In [31]:
avg_time = 0
i = 0

for item in list_users.keys():
    start = pf()
    get_recommended_anime(item, preference_df, top5_peruser_df, anime_df, extensive=True)
    avg_time += (pf() - start)
    print(i)
    i += 1
    
avg_time / 500

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

0.6970005529159662

In [ ]:
idea:
    
instead of rating, given sa tanan crops nga gitanom ni user, asay mas daghan ug nahalin?
expected_yield nga successfully ni planted ni farmer => rating sa anime

- soil based: so if expected - actual (aggregated, over all purchases) 
    is small, recommend => mas suited si crop sa soil profile ni farmer
- business based: if mas daghan ang nahalin sa certain crop, mas suited siya itanom in a business sense
    - dili scientific - dili based sa soil type, bandwagon
    - TBD with ms and carl

intution: if kani nga crop kay mas daghan ug gitanom and nahalin, then 
    
    
* expected = from order
* actual = weighted by atoani from farmer